In [16]:
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import importlib
import warnings

warnings.filterwarnings('ignore')
from tensorflow.keras.models import load_model
import importlib
import Model
importlib.reload(Model)


def get_dataset(path, classes, show_labels):
    for c_idx, c in enumerate(classes):
        raw_data = os.listdir(path+c)
        if len(raw_data) == 1:
            mat = scipy.io.loadmat(path+c+raw_data[0])
        else:
            print("There is more than one dataset - check")

        if c_idx == 0:
            #print(f"Import matlab raw dataset - Matlab file Keys: {mat.keys()}")
            pass

        if mat['Data_Cls'].shape[-1] != mat['Data_Fea'].shape[-1]:
            print(f"Label and dataset do not match! : {len(mat['Data_Cls'])}, {len(mat['Data_Fea'])}")
            break

        if c_idx == 0:
            feature_set = np.transpose(mat['Data_Fea'], (2, 0, 1))  # shape: (1729, 4, 14)
            labels = mat['Data_Cls'].flatten()  #0 or class (either one)

        elif c_idx > 0:
            feature_set_added = np.transpose(mat['Data_Fea'], (2, 0, 1))
            labels_addend = mat['Data_Cls'].flatten()  #0 or class (either one)

            feature_set = np.concatenate([feature_set, feature_set_added], axis=0)
            labels = np.concatenate([labels, labels_addend], axis=0)
        else:
            print("Error in c_idx")
            break

    if show_labels:
        plt.figure(figsize=(8,3))
        plt.plot(labels)
        plt.ylabel('Class label', fontsize=13)
        plt.xlabel('Samples', fontsize=13)
        plt.grid(True, which='both', linestyle='--')
        plt.show()

    return feature_set, labels


def split_data(X, Y, ratio):
    #Splits (N, C, F) data and 1D labels into training and validation sets.

    num_samples = X.shape[0]
    indices = np.random.permutation(num_samples)

    training_size = int(round(num_samples * ratio))
    training_indices = indices[:training_size]
    validation_indices = indices[training_size:]

    training_data = X[training_indices]
    training_labels = Y[training_indices]
    validation_data = X[validation_indices]
    validation_labels = Y[validation_indices]

    training_data = np.expand_dims(training_data, axis=-1)  # Adds a new channel dimension at the end
    validation_data = np.expand_dims(validation_data, axis=-1)

    return training_data, training_labels, validation_data, validation_labels


def train_model(path, classes, train_ratio):
    feature_set, labels = get_dataset(path, classes, show_labels=False)
    X_train, y_train, X_test, y_test = split_data(feature_set, labels, ratio=train_ratio)
    CNN_decoder = Model.Original_model_V1(X_train.shape[1:])

    history, CNN_decoder = Model.Train_model(CNN_decoder, X_train, y_train, X_test, y_test,
                      set_epoch=50, set_batch_size=256, version='V0', set_verbose=False, save_model_set=True)
    acc = CNN_decoder.evaluate(X_test, y_test, verbose=0)[1]
    print(f"Accuracy of test dataset using model V0 (trained with only 5/27 data): {acc*100:.4f}%")

    return CNN_decoder

def testing_new_session(path, classes, train_ratio):
    feature_set, labels = get_dataset(path, classes, show_labels=False)
    _, _, X_test, y_test = split_data(feature_set, labels, ratio=train_ratio)
    acc = CNN_decoder.evaluate(X_test, y_test, verbose=0)[1]
    print(f"Accuracy of unseen dataset using model V0 (trained with only 5/27 data): {acc*100:.4f}%")

    return float(np.round(acc*100, 2))

In [2]:
default_path = "C:/Users/hml76/OneDrive/문서/MATLAB/data/"
classes = ['1/','2/','3/','4/','5/']
train_ratio=0

Info = ['Stand (5/27)', 'Stand (6/18)', 'Stand (6/20)', 'Sit_chair (6/20)', 'Sit_chair_leg_crossed (6/20)', 'Sit_floor (6/20)',
        'Stand (6/20-v2)', 'Sit_chair (6/20-v2)', 'Sit_chair_leg_crossed (6/20-v2)', 'Sit_floor (6/20-v2)']

In [4]:
'''session_info = "Exp_2025-05-27"
path = default_path + f'{session_info}/E8331D05289A/raw/'
CNN_decoder = train_model(path, classes, 0.8)'''

Start Training (total epochs: 50)...
Finish Training! (Model is saved)
Maximum training accuracy : 86.9%
Maximum validation accuracy : 92.75%
Accuracy of seen dataset using model V0 (trained with only 5/27 data): 92.7536%


In [3]:
version = 'V0'
CNN_decoder = load_model(f'best_model_{version}.keras')

In [17]:
info_set = ['Exp_2025-06-18', 'Exp_2025-06-20-v1', 'Exp_2025-06-20-v2', 'Exp_2025-06-20-v3', 'Exp_2025-06-20-v4', 'Exp_2025-06-20-v5', 'Exp_2025-06-20-v6', 'Exp_2025-06-20-v7', 'Exp_2025-06-20-v8']
LST = []

for session_info in info_set:
    path = default_path + f'{session_info}/E9AD0E7DCC2B/raw/'
    acc = testing_new_session(path, classes, train_ratio)
    LST.append(acc)

Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 84.2594%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 79.2734%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 86.0444%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 84.0380%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 68.3814%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 74.5169%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 80.3243%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 76.4644%
Accuracy of unseen dataset using model V0 (trained with only 5/27 data): 74.6695%


In [18]:
LST

[84.26, 79.27, 86.04, 84.04, 68.38, 74.52, 80.32, 76.46, 74.67]